In [3]:
import requests
import pandas as pd
import datetime
import json
import re
from amz_local_search.resources.fba_centers import fba_centers_info

In [2]:
ZONTRACKER_API_USERNAME = 'amzjet'
ZONTRACKER_API_PASSWORD = '84K41VH1Uj'

In [13]:
import bb_utils.time.utils as tu

email = 'dmyers@thatswhatshesaidgame.com'

start_date_s = tu.utc_time('11-01-2019').strftime('%Y-%m-%d')
end_date_s = tu.utc_time('12-28-2019').strftime('%Y-%m-%d')

inventory_api_url_template = 'https://app.zontracker.io/api/inventory/%s/%s/%s'
inventory_api_url = inventory_api_url_template % (email, start_date_s,
                                                  end_date_s)

In [14]:
session = requests.Session()
session.auth = (ZONTRACKER_API_USERNAME, ZONTRACKER_API_PASSWORD)

api_response = session.get(inventory_api_url)
api_response.raise_for_status()
raw_inventory_data = json.loads(api_response.content)

In [25]:
inventory_df = pd.DataFrame(raw_inventory_data.get('data', []))
inventory_df.head(10)

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp
0,SELLABLE,B07H3D54X1,MCO1,That's What She Said Game - Second Expansion,3,A1PTSADUWNXGOK,TWSSXP2,1572764400
1,SELLABLE,X0013VOM91,DTW1,That's What She Said - The Party Game of Twist...,1575,A1PTSADUWNXGOK,TWSSG1,1572764400
2,SELLABLE,X0013VOM91,SLC1,That's What She Said - The Party Game of Twist...,1090,A1PTSADUWNXGOK,TWSSG1,1572764400
3,SELLABLE,X0013VOM91,DEN3,That's What She Said - The Party Game of Twist...,976,A1PTSADUWNXGOK,TWSSG1,1572764400
4,SELLABLE,X0013VOM91,CLE3,That's What She Said - The Party Game of Twist...,89,A1PTSADUWNXGOK,TWSSG1,1572764400
5,SELLABLE,X0013VOM91,RIC2,That's What She Said - The Party Game of Twist...,1301,A1PTSADUWNXGOK,TWSSG1,1572764400
6,SELLABLE,X0013VOM91,EWR9,That's What She Said - The Party Game of Twist...,1022,A1PTSADUWNXGOK,TWSSG1,1572764400
7,SELLABLE,X0013VOM91,CAE1,That's What She Said - The Party Game of Twist...,168,A1PTSADUWNXGOK,TWSSG1,1572764400
8,SELLABLE,X0013VOM91,BFI4,That's What She Said - The Party Game of Twist...,1304,A1PTSADUWNXGOK,TWSSG1,1572764400
9,SELLABLE,B06XCXFBWC,DCA1,That's What She Said Game - First Expansion,19,A1PTSADUWNXGOK,TWSSXP1,1572764400


In [51]:
inventory_df['state'] = inventory_df['fulfillment_center_id'].map(lambda s: '*TRANSFER' if s == '*XFR' else fba_centers_info.code_to_state(s))
inventory_df['date'] = inventory_df['timestamp'].map(lambda t: tu.from_unix_seconds(t))
inventory_df['date_week'] = inventory_df['date'].dt.to_period('W')
inventory_df.head(1)

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp,state,date,date_week
0,SELLABLE,B07H3D54X1,MCO1,That's What She Said Game - Second Expansion,3,A1PTSADUWNXGOK,TWSSXP2,1572764400,FL,2019-11-03 07:00:00+00:00,2019-10-28/2019-11-03


In [52]:
inventory_df[inventory_df.state.map(lambda s: s is None or len(s) != 2)]['fulfillment_center_id'].unique()

array([u'*XFR', u'MAN3', u'LTN1', u'LCY2', u'EUK5', u'MAN1', u'LTN2',
       u'BHX1', u'EDI4', u'GLA1', u'LTN4', u'BRS1', u'EMA1', u'LBA2',
       u'CWL1', u'MAN2', u'BHX4', u'HAM2'], dtype=object)

In [53]:
latest_per_group = inventory_df.groupby(['fnsku', 'fulfillment_center_id', 'state', 'date_week']).agg({'quantity': 'last'})
result = latest_per_group.reset_index().groupby(['state', 'date_week'])['quantity'].sum()
z = result.to_frame().unstack().sort_index().fillna(0)
z

quantity                                              \
date_week 2019-10-28/2019-11-03 2019-11-04/2019-11-10 2019-11-11/2019-11-17   
state                                                                         
*TRANSFER                 738.0                8200.0                2933.0   
AZ                       2322.0                2466.0                2460.0   
CA                       4298.0                5670.0                5932.0   
CO                       1411.0                2295.0                2158.0   
CT                       2843.0                2075.0                1869.0   
DE                         27.0                  26.0                  18.0   
FL                       4713.0                4579.0                4754.0   
GA                          0.0                   0.0                   0.0   
IL                        846.0                 856.0                 789.0   
IN                        539.0                 525.0                 493.0   
KS                       1774.0                2472.0                3053.0   
KY                          2.0                   6.0                   3.0   
MA                          0.0                   1.0                   1.0   
MD                       5409.0                4993.0                4900.0   
MI                       1784.0                1711.0                2044.0   
MN                       1026.0                 999.0                1420.0   
MO                        201.0                 142.0                 216.0   
NC                        553.0                 447.0                 452.0   
NJ                       2861.0                3560.0                3883.0   
NV                         37.0                  28.0                  28.0   
NY                       2787.0                2636.0                2333.0   
OH                       2002.0                3147.0                4230.0   
OK                        122.0                 198.0                 223.0   
OR                        816.0                1200.0                1144.0   
PA                          1.0                   1.0                   1.0   
SC                        172.0                 161.0                 141.0   
TN                         18.0                  21.0                  21.0   
TX                       5746.0                4207.0                7641.0   
UT                       1379.0                1776.0                1656.0   
VA                       1325.0                1308.0                2184.0   
WA                       1568.0                1607.0                1558.0   
WI                       5720.0                 537.0                5494.0   

                                                                             \
date_week 2019-11-18/2019-11-24 2019-11-25/2019-12-01 2019-12-02/2019-12-08   
state                                                                         
*TRANSFER               20322.0               16824.0               13980.0   
AZ                       2434.0                2910.0                2763.0   
CA                       8979.0                6837.0                7716.0   
CO                       3089.0                2932.0                2563.0   
CT                       2987.0                2753.0                3844.0   
DE                         17.0                  14.0                  14.0   
FL                       5473.0                4966.0                5605.0   
GA                          0.0                   0.0                   2.0   
IL                        657.0                 835.0                1824.0   
IN                        445.0                 346.0                 269.0   
KS                       2861.0                4082.0                3571.0   
KY                          5.0                   9.0                   6.0   
MA                          1.0                   2.0                   

In [54]:
z.to_csv('dmyers_by_state.csv')